In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
import random
from transformers import GenerationConfig
import os
import torch

/home/marem/.pyenv/versions/3.11.10/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv("./output/data_on5.csv")

In [3]:
print(df.shape)
df = df[df["cluster_5_nome_ale"] != 'None/Doubt']
df.shape

(2277, 35)


(2245, 35)

In [4]:
print(df.shape)
df = df[df["cluster_5_nome_ale"] != 'None/Doubt']
df.shape

(2245, 35)


(2245, 35)

In [5]:
dict_marem =  {'SonoSfruttatori': "Sono degli sfruttatori",
               'SonoMinaccia': "Sono una minaccia",
               'RovinanoItalia': "Rovinano l'Italia",
               'SonoTutelati': "Sono tutelati",
               'SonoEstremistiReligiosi': "Sono degli estremisti religiosi"}

dict_marco =  {'SonoParassiti': "Sono dei parassiti",
               'SonoSubdoli': "Sono subdoli",
               'SonoImmorali': "Sono immorali",
               'SonoIncompatibiliConNoi': "Sono incompatibili con noi",
               'SonoProblema': "Sono un problema"}

dict_ale = {'FannoQuelloCheVoglionoSenzaContribuire':  "Fanno quello che vogliono senza contribuire",
            'SonoPericolosi': "Sono pericolosi",
            'PeggioranoLeNostreCondizioniDiVita': "Peggiorano le nostre condizioni di vita",
            'HannoCulturaDiversaDallaNostra': "Hanno una cultura diversa dalla nostra",
            'PortanoDegrado': "Portano degrado"}

In [6]:
df["cleaned_cl_marem"] = df["cluster_5_nome_marem"].map(dict_marem)
df["cleaned_cl_marco"] = df["cluster_5_nome_marco"].map(dict_marco)
df["cleaned_cl_ale"] = df["cluster_5_nome_ale"].map(dict_ale)


df = df [['id', 'annotatore', 'tweet', 'chunk', 'annotazione','annotazioni_parsate',
          'cleaned_cl_marem', 'cleaned_cl_marco', 'cleaned_cl_ale']]

In [7]:
dataset = df.sample(n=50)
dataset.columns

Index(['id', 'annotatore', 'tweet', 'chunk', 'annotazione',
       'annotazioni_parsate', 'cleaned_cl_marem', 'cleaned_cl_marco',
       'cleaned_cl_ale'],
      dtype='object')

In [8]:
def create_prompt(row):

  text = row["tweet"]
  opt_marem = row['cleaned_cl_marem']
  opt_marco = row['cleaned_cl_marco']
  opt_ale = row['cleaned_cl_ale']
  random.seed(42)
  list_options = [opt_marem, opt_marco, opt_ale]
  random.shuffle(list_options)

  instruction = {"prelude": "Ti viene fornita in input (Input) una frase estratta dai social media, insieme a tre possibili stereotipi (Opzioni).",
        "task": "Il tuo compito è individuare quale stereotipo è implicito nella frase, scegliendo tra le opzioni fornite." ,
        "instr": "Restituisci in output (Output) una singola opzione, sotto forma di lista Python (es. ['Opzione 1']).",
        "input": f"Input: {text}",
        "options": f"Opzioni: {list_options}",
        "output": "Output:"}

  prompt = f"{instruction['prelude']} {instruction['task']} {instruction['instr']}\n {instruction['input']}\n {instruction['options']}\n {instruction['output']}"

  return prompt



dataset["prompt"] = dataset.apply(create_prompt, axis=1)

In [9]:
model_id = "sapienzanlp/Minerva-7B-instruct-v1.0"

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

KeyboardInterrupt: 